### Install enoslib (+ small setup)

In [1]:
#!pip install -U pip
#!pip install -y enoslib
#!pip install python-grid5000
ingi = True
if ingi:
    !PYTHONPATH=/etinfo/users/2020/adgiot/.local/lib/python3.10/site-packages/

In [ ]:
# need for credentials in .env
#!echo -e "username: $(head -n 1 .env)\npassword: $(head -n 2 .env | tail -n 1)" > ~/.python-grid5000.yaml

In [2]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

[WARNING]: failed to patch stdout/stderr for fork-safety: 'OutStream' object
has no attribute 'buffer'


[<Site uid:grenoble>,
 <Site uid:lille>,
 <Site uid:luxembourg>,
 <Site uid:lyon>,
 <Site uid:nancy>,
 <Site uid:nantes>,
 <Site uid:rennes>,
 <Site uid:sophia>,
 <Site uid:toulouse>]

### Launch machines

In [3]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

job_name = 'dev'

conf = (
    en.G5kConf.from_settings(
        job_name=job_name,
        job_type=[],#"deploy"],
        key="/home/adrien/.ssh/grid",
        #env_name="ubuntu2204-min",
        walltime="5:00:00"
    )    
    #.add_machine(roles=["client"], cluster="uvb", nodes=1)
    #.add_machine(roles=["server"], cluster="uvb", nodes=1)
    .add_machine(roles=["client"], servers=["uvb-27.sophia.grid5000.fr"]) #cluster="grimoire", nodes=1)
    .add_machine(roles=["server"], servers=["uvb-38.sophia.grid5000.fr"]) #cluster="grimoire", nodes=1)
    #.add_machine(roles=["client"], servers=["grimoire-3.nancy.grid5000.fr"]) #cluster="grimoire", nodes=1)
    #.add_machine(roles=["server"], servers=["grimoire-4.nancy.grid5000.fr"]) #cluster="grimoire", nodes=1)
)

provider = en.G5k(conf)

roles, networks = provider.init()

Output()

Finished 1 tasks (Granting root access on the nodes (sudo-g5k)) on 
{'uvb-27.sophia.grid5000.fr', 'uvb-38.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

In [5]:
provider.jobs

[<Job uid:4016744 site:nancy state:running user:agiot>]

### Setup the machines

In [10]:
# get memcache server running on server (localhost port 8000)
results = en.run_command("sudo-g5k apt install -y memcached libmemcached-tools", roles=roles["server"])

#results = en.run_command("memcached -p 11211 -l 172.18.132.38 -d", roles=roles["server"], background=True)
# run manually for now

# telnet 127.0.0.1/172.18.132.38 11211

Output()

Finished 1 tasks (sudo-g5k apt install -y memcached libmemcached-tools) on 
{'uvb-38.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

In [7]:
# install memcache client 
results = en.run_command('git clone https://github.com/RedisLabs/memtier_benchmark', roles=roles["client"])
results = en.run_command('cd memtier_benchmark && autoreconf -ivf && ./configure && make && sudo-g5k make install', roles=roles["client"])
# memtier_benchmark --help

Output()

Finished 1 tasks (git clone https://github.com/RedisLabs/memtier_benchmark) on 
{'uvb-27.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (cd memtier_benchmark && autoreconf -ivf && ./configure && make && sudo-g5k 
make install) on {'uvb-27.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

In [10]:
for r in results:
    print(r)

CommandResult(host='uvb-38.sophia.grid5000.fr', task='cd memtier_benchmark; autoreconf -ivf; 
./configure; make; sudo-g5k make install', status='OK', payload={'changed': True, 'stdout': 
"libtoolize: putting auxiliary files in '.'.\nlibtoolize: copying file 
'./ltmain.sh'\nlibtoolize: putting macros in AC_CONFIG_MACRO_DIRS, 'm4'.\nlibtoolize: copying
file 'm4/libtool.m4'\nlibtoolize: copying file 'm4/ltoptions.m4'\nlibtoolize: copying file 
'm4/ltsugar.m4'\nlibtoolize: copying file 'm4/ltversion.m4'\nlibtoolize: copying file 
'm4/lt~obsolete.m4'\nlibtoolize: Consider adding '-I m4' to ACLOCAL_AMFLAGS in 
Makefile.am.\nchecking for a BSD-compatible install... /usr/bin/install -c\nchecking whether 
build environment is sane... yes\nchecking for a thread-safe mkdir -p... /usr/bin/mkdir 
-p\nchecking for gawk... gawk\nchecking whether make sets $(MAKE)... yes\nchecking whether 
make supports nested variables... yes\nchecking whether make supports nested variables... 
(cached) yes\nchecking build system type... x86_64-pc-linux-gnu\nchecking host system type...
x86_64-pc-linux-gnu\nchecking how to print strings... printf\nchecking whether make supports 
the include directive... yes (GNU style)\nchecking for gcc... gcc\nchecking whether the C 
compiler works... yes\nchecking for C compiler default output file name... a.out\nchecking 
for suffix of executables... \nchecking whether we are cross compiling... no\nchecking for 
suffix of object files... o\nchecking whether we are using the GNU C compiler... 
yes\nchecking whether gcc accepts -g... yes\nchecking for gcc option to accept ISO C89... 
none needed\nchecking whether gcc understands -c and -o together... yes\nchecking dependency 
style of gcc... gcc3\nchecking for a sed that does not truncate output... 
/usr/bin/sed\nchecking for grep that handles long lines and -e... /usr/bin/grep\nchecking for
egrep... /usr/bin/grep -E\nchecking for fgrep... /usr/bin/grep -F\nchecking for ld used by 
gcc... /usr/bin/ld\nchecking if the linker (/usr/bin/ld) is GNU ld... yes\nchecking for BSD- 
or MS-compatible name lister (nm)... /usr/bin/nm -B\nchecking the name lister (/usr/bin/nm 
-B) interface... BSD nm\nchecking whether ln -s works... yes\nchecking the maximum length of 
command line arguments... 1572864\nchecking how to convert x86_64-pc-linux-gnu file names to 
x86_64-pc-linux-gnu format... func_convert_file_noop\nchecking how to convert 
x86_64-pc-linux-gnu file names to toolchain format... func_convert_file_noop\nchecking for 
/usr/bin/ld option to reload object files... -r\nchecking for objdump... objdump\nchecking 
how to recognize dependent libraries... pass_all\nchecking for dlltool... no\nchecking how to
associate runtime and link libraries... printf %s\\n\nchecking for ar... ar\nchecking for 
archiver @FILE support... @\nchecking for strip... strip\nchecking for ranlib... 
ranlib\nchecking command to parse /usr/bin/nm -B output from gcc object... ok\nchecking for 
sysroot... no\nchecking for a working dd... /usr/bin/dd\nchecking how to truncate binary 
pipes... /usr/bin/dd bs=4096 count=1\nchecking for mt... mt\nchecking if mt is a manifest 
tool... no\nchecking how to run the C preprocessor... gcc -E\nchecking for ANSI C header 
files... yes\nchecking for sys/types.h... yes\nchecking for sys/stat.h... yes\nchecking for 
stdlib.h... yes\nchecking for string.h... yes\nchecking for memory.h... yes\nchecking for 
strings.h... yes\nchecking for inttypes.h... yes\nchecking for stdint.h... yes\nchecking for 
unistd.h... yes\nchecking for dlfcn.h... yes\nchecking for objdir... .libs\nchecking if gcc 
supports -fno-rtti -fno-exceptions... no\nchecking for gcc option to produce PIC... -fPIC 
-DPIC\nchecking if gcc PIC flag -fPIC -DPIC works... yes\nchecking if gcc static flag -static
works... yes\nchecking if gcc supports -c -o file.o... yes\nchecking if gcc supports -c -o 
file.o... (cached) yes\nchecking whether the gcc linker (/usr/bin/ld -m elf_x86_64) supports 
shared libraries... y

In [12]:
# rdma proxies
results = en.run_command("cd /home/agiot/infinity; make library; make examples", roles=roles["server"])
for result in results:
    #print(f"{result.host} has {result.payload['stdout']} logical CPU cores")
    print(result.payload['stdout'])

Output()

Finished 1 tasks (cd /home/agiot/infinity; make library; make examples) on 
{'uvb-38.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

mkdir -p release
ar rvs release/libinfinity.a build/infinity/core/Context.o build/infinity/memory/Atomic.o build/infinity/memory/Buffer.o build/infinity/memory/Region.o build/infinity/memory/RegionToken.o build/infinity/memory/RegisteredMemory.o build/infinity/queues/QueuePair.o build/infinity/queues/QueuePairFactory.o build/infinity/requests/RequestToken.o build/infinity/utils/Address.o
r - build/infinity/core/Context.o
r - build/infinity/memory/Atomic.o
r - build/infinity/memory/Buffer.o
r - build/infinity/memory/Region.o
r - build/infinity/memory/RegionToken.o
r - build/infinity/memory/RegisteredMemory.o
r - build/infinity/queues/QueuePair.o
r - build/infinity/queues/QueuePairFactory.o
r - build/infinity/requests/RequestToken.o
r - build/infinity/utils/Address.o
rm -rf release/include
cp --parents src/infinity/infinity.h src/infinity/core/Context.h src/infinity/core/Configuration.h src/infinity/memory/Atomic.h src/infinity/memory/Buffer.h src/infinity/memory/Region.h src/infinity/me

In [13]:
# tcp proxies
results = en.run_command("cd /home/agiot/tcp-echo; g++ -O3 -pthread -o client.o client.cpp; g++ -O3 -pthread -o server.o server.cpp; ", roles=roles["server"])
for result in results:
    #print(f"{result.host} has {result.payload['stdout']} logical CPU cores")
    print(result.payload['stdout'])

Output()

Finished 1 tasks (cd /home/agiot/tcp-echo; g++ -O3 -pthread -o client.o client.cpp; g++ -O3 
-pthread -o server.o server.cpp; ) on {'uvb-38.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

### Run the experiments

In [21]:
import time

results_s = en.run_command("cd /home/agiot/tcp-echo; ./server.o", roles=roles["server"], background=True)
results_c = en.run_command("cd /home/agiot/tcp-echo; ./client.o", roles=roles["client"])

for result in results_c:
    if 'failed' in result.stdout:
        print('socket aint killed yet')
        time.sleep(5)
    print(result.stdout)


Output()

Finished 1 tasks (cd /home/agiot/tcp-echo; ./server.o) on {'grimoire-4.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (cd /home/agiot/tcp-echo; ./client.o) on {'grimoire-3.nancy.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Socket successfully created..
connected to the server..
Socket successfully created..
Socket successfully binded..
Server listening..
server accept the client...
it's fine
397
261
195
261
303
281
167
206
305
197
215
178
175
284
279
243
280
280
155
225
283
169
162
161
211
133
313
228
243
252
298
241
249
309
284
284
274
264
169
231
207
271
300
184
226
304
221
258
284
232
164
186
271
241
257
306
232
247
291
217
306
289
301
234
185
263
304
261
275
195
207
195
217
197
203
185
188
182
316
236
255
120
211
120
300
231
313
272
302
207
181
310
193
304
240
167
205
281
222
192
202
161
185
163
265
270
281
236
177
203
262
237
263
305
259
274
272
246
191
302
293
127
158
293
259
164
184
116
257
166
298
271
184
192
198
154
260
297
276
270
286
207
92
268
259
265
261
195
125
162
295
122
206
199
170
205
255
229
246
246
229
157
171
298
174
286
254
299
287
196
303
239
285
265
159
171
249
204
147
149
151
156
292
266
297
268
158
154
173
253
259
300
246
257
192
197
313
206
224
262
210
177
259
284
172
195
147
2

In [21]:
result.stdout.split('\n')[7:]

['372',
 '320',
 '313',
 '255',
 '320',
 '267',
 '342',
 '131',
 '222',
 '190',
 '264',
 '245',
 '251',
 '315',
 '227',
 '341',
 '273',
 '262',
 '296',
 '326',
 '349',
 '256',
 '318',
 '341',
 '274',
 '298',
 '320',
 '227',
 '298',
 '310',
 '284',
 '250',
 '350',
 '339',
 '288',
 '338',
 '235',
 '232',
 '268',
 '335',
 '247',
 '345',
 '267',
 '327',
 '324',
 '337',
 '329',
 '319',
 '310',
 '318',
 '308',
 '288',
 '325',
 '243',
 '256',
 '319',
 '238',
 '286',
 '255',
 '322',
 '250',
 '314',
 '335',
 '237',
 '330',
 '258',
 '235',
 '212',
 '239',
 '267',
 '309',
 '292',
 '321',
 '257',
 '337',
 '313',
 '338',
 '305',
 '243',
 '253',
 '350',
 '322',
 '334',
 '253',
 '321',
 '333',
 '295',
 '255',
 '252',
 '174',
 '301',
 '344',
 '298',
 '255',
 '319',
 '325',
 '286',
 '313',
 '319',
 '315',
 '318',
 '344',
 '275',
 '322',
 '341',
 '262',
 '340',
 '330',
 '326',
 '271',
 '360',
 '236',
 '237',
 '297',
 '343',
 '286',
 '339',
 '325',
 '256',
 '242',
 '318',
 '296',
 '335',
 '314',
 '343',


In [15]:
#_ = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send -s", roles=roles["server"], background=True)
#results_c = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send", roles=roles["client"])
for result in results_c:
    print(result.stdout.split('\n')[4:])

['118', '42', '37', '35', '43', '34', '35', '34', '39', '51', '37', '37', '35', '35', '35', '34', '39', '35', '36', '35', '33', '9', '12', '35', '36', '35', '35', '33', '36', '35', '36', '34', '42', '35', '33', '33', '66', '34', '35', '34', '34', '33', '34', '17', '36', '34', '34', '35', '34', '36', '41', '34', '34', '33', '32', '33', '33', '12', '33', '33', '35', '42', '33', '34', '20', '34', '34', '33', '32', '49', '32', '13', '33', '34', '33', '33', '33', '32', '57', '34', '34', '33', '32', '33', '32', '33', '35', '34', '32', '31', '33', '31', '34', '34', '34', '33', '32', '31', '33', '38', '10', '33', '32', '32', '34', '33', '35', '33', '32', '32', '31', '31', '32', '32', '32', '31', '32', '31', '30', '31', '32', '31', '31', '31', '30', '30', '31', '31', '40', '32', '32', '32', '31', '32', '32', '31', '33', '31', '32', '31', '47', '31', '31', '31', '31', '31', '32', '17', '31', '31', '32', '32', '31', '31', '14', '32', '32', '31', '30', '31', '32', '15', '31', '32', '32', '30', '31

In [18]:
_ = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send -s", roles=roles["server"], background=True)
_ = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send", roles=roles["client"], background=True)
results_c = en.run_command("cd /home/agiot/memtier_benchmark; ./memtier_benchmark -s 127.0.0.1 -p 8084 -P memcache_text -c 1 -t 1", roles=roles["client"])
_ = en.run_command("sudo kill $(sudo lsof -t -i:8020)", roles=roles["client"])
_ = en.run_command("sudo kill $(sudo lsof -t -i:8020)", roles=roles["server"])
for result in results_c:
    cur_echo_times = result.stdout #.split('\n')
    print(cur_echo_times)

Output()

Finished 1 tasks (cd /home/agiot/infinity; ./release/examples/read-write-send -s) on 
{'uvb-38.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (cd /home/agiot/infinity; ./release/examples/read-write-send) on 
{'uvb-27.sophia.grid5000.fr'}

...

Output()

Finished 1 tasks (sudo kill $(sudo lsof -t -i:8020)) on {'uvb-38.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

1         Threads
1         Connections per thread
10000     Requests per client


ALL STATS
=============================================================================================
===============================
Type         Ops/sec     Hits/sec   Misses/sec    Avg. Latency     p50 Latency     p99 
Latency   p99.9 Latency       KB/sec 
---------------------------------------------------------------------------------------------
-------------------------------
Sets          404.38          ---          ---         0.22628         0.22300         
0.30300         0.31100        27.59 
Gets         4039.34         0.00      4039.34         0.22243         0.22300         
0.29500         0.31100       102.12 
Waits           0.00          ---          ---             ---             ---             
---             ---          --- 
Totals       4443.71         0.00      4039.34         0.22278         0.22300         
0.29500         0.31100       129.71 


Request Latency Distribution
Type     <= msec         Percent
------------------------------------------------------------------------
SET       0.151        0.000
SET       0.191        5.000
SET       0.191       10.000
SET       0.199       15.000
SET       0.207       20.000
SET       0.207       25.000
SET       0.215       30.000
SET       0.215       35.000
SET       0.215       40.000
SET       0.223       45.000
SET       0.223       50.000
SET       0.231       52.500
SET       0.231       55.000
SET       0.231       57.500
SET       0.231       60.000
SET       0.239       62.500
SET       0.239       65.000
SET       0.239       67.500
SET       0.239       70.000
SET       0.247       72.500
SET       0.247       75.000
SET       0.247       76.250
SET       0.247       77.500
SET       0.255       78.750
SET       0.255       80.000
SET       0.255       81.250
SET       0.255       82.500
SET       0.255       83.750
SET       0.255       85.000
SET       0.263       86.250
SET       0.263       87.500
SET       0.263       88.125
SET       0.263       88.750
SET       0.263       89.375
SET       0.263       90.000
SET       0.271       90.625
SET       0.271       91.250
SET       0.271       91.875
SET       0.271       92.500
SET       0.279       93.125
SET       0.279       93.750
SET       0.279       94.062
SET       0.279       94.375
SET       0.279       94.688
SET       0.279       95.000
SET       0.279       95.312
SET       0.279       95.625
SET       0.279       95.938
SET       0.287       96.250
SET       0.287       96.562
SET       0.287       96.875
SET       0.287       97.031
SET       0.287       97.188
SET       0.287       97.344
SET       0.287       97.500
SET       0.287       97.656
SET       0.295       97.812
SET       0.295       97.969
SET       0.295       98.125
SET       0.303       98.281
SET       0.303       98.438
SET       0.303       98.516
SET       0.303       98.594
SET       0.303       98.672
SET       0.303       98.750
SET       0.303       98.828
SET       0.303       98.906
SET       0.303       98.984
SET       0.303       99.062
SET       0.303       99.141
SET       0.303       99.219
SET       0.303       99.258
SET       0.303       99.297
SET       0.303       99.336
SET       0.303       99.375
SET       0.303       99.414
SET       0.311       99.453
SET       0.311       99.492
SET       0.311       99.531
SET       0.311       99.570
SET       0.311       99.609
SET       0.311       99.629
SET       0.311       99.648
SET       0.311       99.668
SET       0.311       99.688
SET       0.311       99.707
SET       0.311       99.727
SET       0.311       99.746
SET       0.311       99.766
SET       0.311       99.785
SET       0.311       99.805
SET       0.311       99.814
SET       0.311       99.824
SET       0.311       99.834
SET       0.311       99.844
SET       0.311       99.854
SET       0.311       99.863
SET       0.311       99.873
SET       0.311       99.883
SET       0.503  

RecursionError: maximum recursion depth exceeded

In [35]:
results_c = en.run_command("cd /home/agiot/memtier_benchmark; ./memtier_benchmark -s 172.18.132.38 -p 11211 -P memcache_text -c 1 -t 1", roles=roles["client"])
for result in results_c:
    cur_echo_times = result.stdout
    print(cur_echo_times)

Output()

Finished 1 tasks (cd /home/agiot/memtier_benchmark; ./memtier_benchmark -s 172.18.132.38 -p 
11211 -P memcache_text -c 1 -t 1) on {'uvb-27.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

1         Threads
1         Connections per thread
10000     Requests per client


ALL STATS
Type         Ops/sec     Hits/sec   Misses/sec    Avg. Latency     p50 Latency     p99 Latency   p99.9 Latency       KB/sec 
----------------------------------------------------------------------------------------------------------------------------
Sets          329.62          ---          ---         0.27736         0.27100         0.39900         0.42300        22.49 
Gets         3292.60         0.00      3292.60         0.27207         0.26300         0.39900         0.42300        83.24 
Waits           0.00          ---          ---             ---             ---             ---             ---          --- 
Totals       3622.22         0.00      3292.60         0.27255         0.26300         0.39900         0.42300       105.73 


Request Latency Distribution
Type     <= msec         Percent
------------------------------------------------------------------------
SET       0.183     

In [ ]:
results_c = en.run_command("cd /home/agiot/memtier_benchmark; ./memtier_benchmark -s  -p 8084 -P memcache_text -c 1 -t 1", roles=roles["client"])

In [28]:
for result in results_c:
    cur_echo_times = result.stdout
    print(cur_echo_times)

1         Threads
1         Connections per thread
10000     Requests per client


ALL STATS
Type         Ops/sec     Hits/sec   Misses/sec    Avg. Latency     p50 Latency     p99 Latency   p99.9 Latency       KB/sec 
----------------------------------------------------------------------------------------------------------------------------
Sets            0.00          ---          ---             ---             ---             ---             ---         0.00 
Gets            0.00         0.00         0.00             ---             ---             ---             ---         0.00 
Waits           0.00          ---          ---             ---             ---             ---             ---          --- 
Totals          0.00         0.00         0.00            -nan         0.00000         0.00000         0.00000         0.00 


Request Latency Distribution
Type     <= msec         Percent
------------------------------------------------------------------------
SET       0.000     

In [37]:
rdma_times = []
tcp_times = []
nop_times = []
for _ in range(5):
    
    results_c = en.run_command("cd /home/agiot/memtier_benchmark; ./memtier_benchmark -s 172.18.132.38 -p 11211 -P memcache_text -c 1 -t 1", roles=roles["client"])
    for result in results_c:
        cur_times = result.stdout
        nop_times.append(cur_times)
        
    _ = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send -s", roles=roles["server"], background=True)
    _ = en.run_command("cd /home/agiot/infinity; ./release/examples/read-write-send", roles=roles["client"], background=True)
    results_c = en.run_command("cd /home/agiot/memtier_benchmark; ./memtier_benchmark -s 127.0.0.1 -p 8084 -P memcache_text -c 1 -t 1", roles=roles["client"])
    _ = en.run_command("sudo kill $(sudo lsof -t -i:8020)", roles=roles["client"])
    _ = en.run_command("sudo kill $(sudo lsof -t -i:8020)", roles=roles["server"])
    for result in results_c:
        cur_times = result.stdout #.split('\n')
        rdma_times.append(cur_times)
    
    
    finished = False
    while not finished:
        _ = en.run_command("cd /home/agiot/tcp-echo; ./server.o", roles=roles["server"], background=True)
        _ = en.run_command("cd /home/agiot/tcp-echo; ./client.o", roles=roles["client"], background=True)
        results_c = en.run_command("cd /home/agiot/memtier_benchmark; ./memtier_benchmark -s 127.0.0.1 -p 8084 -P memcache_text -c 1 -t 1", roles=roles["client"])
        finished = True
        try:
            _ = en.run_command("sudo kill $(sudo lsof -t -i:8085)", roles=roles["client"])
            _ = en.run_command("sudo kill $(sudo lsof -t -i:8086)", roles=roles["server"])
        except:
            finished = False
            continue
        for result in results_c:
            cur_times = result.stdout #.split('\n')[7:]
            tcp_times.append(cur_times)            
    
rdma_echo_times
tcp_echo_times
nop_times

Output()

Finished 1 tasks (cd /home/agiot/memtier_benchmark; ./memtier_benchmark -s 172.18.132.38 -p 
11211 -P memcache_text -c 1 -t 1) on {'uvb-27.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (cd /home/agiot/infinity; ./release/examples/read-write-send -s) on 
{'uvb-38.sophia.grid5000.fr'}

...

─────────────────────────────────────────────────────────────────────────────────────────────

Output()

Finished 1 tasks (sudo kill $(sudo lsof -t -i:8086)) on {'uvb-38.sophia.grid5000.fr'}

─────────────────────────────────────────────────────────────────────────────────────────────

['1         Threads\n1         Connections per thread\n10000     Requests per client\n\n\nALL STATS\n============================================================================================================================\nType         Ops/sec     Hits/sec   Misses/sec    Avg. Latency     p50 Latency     p99 Latency   p99.9 Latency       KB/sec \n----------------------------------------------------------------------------------------------------------------------------\nSets          317.69          ---          ---         0.28590         0.27900         0.40700         0.42300        21.68 \nGets         3173.38         0.00      3173.38         0.28241         0.27900         0.39900         0.42300        80.23 \nWaits           0.00          ---          ---             ---             ---             ---             ---          --- \nTotals       3491.07         0.00      3173.38         0.28273         0.27900         0.39900         0.42300       101.91 \n\n\nRequest Laten

In [34]:
rdma_echo_times

['1         Threads\n1         Connections per thread\n10000     Requests per client\n\n\nALL STATS\n============================================================================================================================\nType         Ops/sec     Hits/sec   Misses/sec    Avg. Latency     p50 Latency     p99 Latency   p99.9 Latency       KB/sec \n----------------------------------------------------------------------------------------------------------------------------\nSets          356.57          ---          ---         0.26204         0.25500         0.39100         0.43100        24.33 \nGets         3561.83         0.00      3561.83         0.25200         0.24700         0.38300         0.39900        90.05 \nWaits           0.00          ---          ---             ---             ---             ---             ---          --- \nTotals       3918.40         0.00      3561.83         0.25292         0.24700         0.39100         0.39900       114.38 \n\n\nRequest Laten

In [25]:

tcp_echo_times[1]

['Type         Ops/sec     Hits/sec   Misses/sec    Avg. Latency     p50 Latency     p99 Latency   p99.9 Latency       KB/sec ',
 '----------------------------------------------------------------------------------------------------------------------------',
 'Sets            0.00          ---          ---             ---             ---             ---             ---         0.00 ',
 'Gets            0.00         0.00         0.00             ---             ---             ---             ---         0.00 ',
 'Waits           0.00          ---          ---             ---             ---             ---             ---          --- ',
 'Totals          0.00         0.00         0.00            -nan         0.00000         0.00000         0.00000         0.00 ',
 '',
 '',
 'Request Latency Distribution',
 'Type     <= msec         Percent',
 '------------------------------------------------------------------------',
 'SET       0.000      100.000',
 '---',
 'GET       0.000      100.0

In [40]:
import pandas as pd  

to_save = {'rdma':rdma_times,           
           'tcp':tcp_times,
           'nop':nop_times}

df = pd.DataFrame(to_save) 
    
# saving the dataframe 
df.to_csv('memcache.csv')

In [50]:
len(tcp_echo_times[0])


1000

### Kill every last one of them

In [30]:
# Release all Grid'5000 resources
provider.destroy()